In [1]:
%pip install ultralytics
%pip install super-gradients
%pip install supervision
%pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 953.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.3 MB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.6 MB/s eta 0:00:00


In [2]:
from ultralytics import NAS

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from super_gradients.training import models
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

best_model = models.get(
    'yolo_nas_l',
    num_classes=1,
    checkpoint_path="/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth"
).to(DEVICE)


[2024-01-29 14:16:58] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-01-29 14:16:58] WARNING - __init__.py - Failed to import pytorch_quantization


The console stream is logged into /root/sg_logs/console.log


[2024-01-29 14:17:04] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-01-29 14:17:04] WARNING - export.py - Failed to import pytorch_quantization
[2024-01-29 14:17:04] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2024-01-29 14:17:04] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-01-29 14:17:04] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-01-29 14:17:04] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-01-29 14:17:04] WARNING - env_sanity_check.py - Failed to verify installed packages: sphinx-rtd-theme required but not found
[2024-01-29 14:17:04] WARNING - env_sanity_check.py - Failed to verify installed packages: torchmetrics required but not found
[2024-01-29 14:17:04] WARNING - env_sanity_check.py - Failed to verify installed packages: hydra-core 

In [12]:
import supervision as sv
import ultralytics

def get_inference(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )

  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result =best_model.predict(image, conf=CONFIDENCE_TRESHOLD)
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections
    nome_imagem = image_name.split('/')[7] + ".txt"
    with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/inferencias/{nome_imagem}","w") as file:
      file.write(str(detections.xyxy))

get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2024-01-29 14:20:28] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:20:29] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:20:29] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:20:29] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:20:29] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:20:30] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:20:30] INFO -

In [13]:
import supervision as sv
import ultralytics
box_annotator = sv.BoxAnnotator()

images = []
titles = []

detections_list = []
prediction_list = []

def get_ioudata(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )
  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = best_model.predict(image, conf=CONFIDENCE_TRESHOLD)
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

  keys = list(ds.images.keys())
  for key in keys:
      # Para anotações
      annotation_detections = ds.annotations[key]
      for detection in annotation_detections:
          detections_list.append(detection[0])

      # Para previsões
      prediction_detections = predictions[key]
      prediction_list.append(prediction_detections)

get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2024-01-29 14:24:39] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:24:39] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:24:40] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:24:40] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:24:40] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:24:41] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-29 14:24:41] INFO -

In [18]:
import numpy as np


def iou_xyxy(detections, predictions):

  def iou(box1,box2):
    # Check if the bounding boxes are valid.
    if box1[2] is None or box2[2] is None:
      return 0.0

    # Calculate the intersection area.
    inter_top = max(box1[1], box2[1])
    inter_bottom = min(box1[3], box2[3])
    inter_left = max(box1[0], box2[0])
    inter_right = min(box1[2], box2[2])
    inter_area = (inter_right - inter_left) * (inter_bottom - inter_top)

    # Calculate the union area.
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    # Calculate the IoU.
    iou = inter_area / union_area
    return iou


  for box2 in predictions:
    print(detections)
    print(box2[0])
    print(calculate_iou(detections,box2[0]))
    return calculate_iou(detections,box2[0])


for i in range(len(detections_list)):
  try:
    iou_score = iou_xyxy(detections_list[i], prediction_list[i])
  except Exception as e:
    print("fim das previsões", i)
  finally:
    with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/Iou.txt",'a') as file:
      file.write(str(iou_score)+ '\n')


[     214.44       32.22         305      112.22]
[     214.18      31.396      294.17      102.93]
0.7692672
[     263.85      143.85      403.08      293.85]
[      262.8         142       404.6       295.2]
0.96136683
[      75.17       65.52      128.27         120]
[     73.938      64.838       128.8      122.24]
0.9186375
[     106.55       106.9      172.07      181.04]
[     102.38      105.79      168.61      184.27]
0.8434799
[     174.83      108.96      215.17      149.65]
[     173.25      107.97      215.07      151.11]
0.9056891
[     184.62      318.46       242.3      384.61]
[      183.5       314.8       237.2       385.2]
0.8446695
[     202.31      263.85      275.39      393.08]
[      193.3       259.2       279.4         388]
0.7916934
[     143.79      105.51      182.41      138.96]
[      144.9       107.1      182.88       137.9]
0.8845069
[     327.69       272.3      383.07      323.08]
[      325.2       269.6       382.4       327.2]
0.83443916
[     18

In [17]:
def calculate_iou(box1, box2):
    """
    Calcula a interseção sobre união (IoU) de dois bounding boxes no formato xyxy.

    Args:
    - box1 (list): Lista contendo as coordenadas [x1, y1, x2, y2] do primeiro bounding box.
    - box2 (list): Lista contendo as coordenadas [x3, y3, x4, y4] do segundo bounding box.

    Returns:
    - float: Valor da IoU entre os dois bounding boxes.
    """
    # Calcular a interseção
    x_intersection = max(0, min(box1[2], box2[2]) - max(box1[0], box2[0]))
    y_intersection = max(0, min(box1[3], box2[3]) - max(box1[1], box2[1]))
    area_intersection = x_intersection * y_intersection

    # Calcular a união
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    area_union = area_box1 + area_box2 - area_intersection

    # Calcular a IoU
    iou = area_intersection / max(area_union, 1e-10)  # Evitar divisão por zero

    return iou

bx1 =[     107.59,      95.515,      132.07,      113.44]
bx2 =[      135.8,       243.6,       241.6,       367.6]
print(calculate_iou(bx1,bx2))

0.0


In [ ]:

#print(detections.xyxy)

#iou(detections_list[0], prediction_list[0])
print(len(detections_list), len(prediction_list))


315 301
